## GuidedBackprop_CREs-prediction

This code outputs confidence values for prediction of CREs and visualize the residues relevant to the prediction, in a batch system.

INPUT
1. Sequence tiles (originally 31-bp) to visualualize relevance to prediction of TF-binding sites (or CREs).
   NOTE: adjust the shape of sequence tiles to the originally trained one.
2. h5 model trained with cistrome data, which was used for prediction of TF-binding sites (or CREs).

OUTPUT
1. "prediction_test.txt" including confidences in prediction of CREs, for each sequence tiles.
2. Heatmaps of relevant residues in sequence tiles, in ./GBPheatmap_CREs/ directory


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from visualizations_forCisDecode import GradCAM, GuidedGradCAM, GBP, LRP, CLRP, SGLRP, LRPA, LRPB, LRPE
from keras.layers import (Activation, Add, GlobalAveragePooling2D,
                          BatchNormalization, Conv1D, Conv2D, Dense, Flatten, Reshape, Input, Dropout,
                          MaxPooling1D,MaxPooling2D)

from helper_forCisDecode2 import heatmap, heatmap_optional
import innvestigate.utils as iutils
import os
from keras.models import load_model, Model
from keras.utils import plot_model,np_utils
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model

from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras import backend as K
from keras import optimizers

from functools import reduce
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from sklearn.metrics import classification_report, confusion_matrix

import skimage as sk
sk.__version__

# limits tensorflow to a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
### inputs ###

frag = '31bp-fragments.fa' ### 31-bp fragments for relevance propagation
trained_model = "example2.h5" ### h5 model trained with cistrome data, which was used for prediction of TF-binding sites, or CREs


In [ ]:
def dna2num(dna):
    if dna.upper() == "A":
        return 0
    elif dna.upper() == "T":
        return 1
    elif dna.upper() == "G":
        return 2
    else:
        return 3
    
def num2dna(num):
    if num == 0:
        return "A"
    elif num == 1:
        return "T"
    elif num == 2:
        return "G"
    else:
        return "C"

def dna2array(DNAstring):
    numarr = []
    length = len(DNAstring)
    for i in range(0, length): 
        num = dna2num(DNAstring[i:i+1]) 
        if num >= 0:
            numarr.append(num) 
    return numarr

def array2dna(numarr):
    DNAstring = []
    length = numarr.shape[0]
    for i in range(0, length): 
        dna = num2dna(numarr[i].argmax()) 
        DNAstring.append(dna) 
    DNAstring = ''.join(DNAstring)
    return DNAstring

In [ ]:
def load_data():
    X = []
    Y = []
    f = open(frag, "r")
    line=f.readline()
    while line:
        line2 = line.rstrip()
        OneHotArr = np.array([np.eye(4)[dna2array(line2)]])
        X.extend(OneHotArr)
        Y.append(1)
        line = f.readline()
    X = np.array(X)
    Y = np.array(Y)
    Y = np_utils.to_categorical(Y, 2)    
    X = np.reshape(X,(-1, 31, 4, 1))
   
    return (X, Y)

In [ ]:
input_shape=(82055,31,4,1)
num_classes=2
model = load_model(trained_model)
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model.summary())  #modelのsummaryを表示

In [ ]:
partial_model = Model(
    inputs=model.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model.outputs),
    name=model.name,
)
partial_model.summary()

In [ ]:
X, Y = load_data()
orig_imgs  = X.reshape((-1,31,4,1))
orig_names = []
for img in orig_imgs:
    orig_names.append(array2dna(img))
    
predictions = model.predict(orig_imgs)
pred_classes = np.argmax(predictions, axis=1)
target_classes = np.argmax(Y, axis=1)

In [ ]:
### output prediction confidence in "prediction_test.txt" ####

write_file = "./prediction_test.txt"

negaposi = ["nega","posi"]

with open(write_file,'w') as f:
    f.write('prediction[nega, posi]:\n')
    for i in range(len(orig_names)):
        f.write(str(i)+"\t"+str(orig_names[i])+"\t"+str(negaposi[target_classes[i]])+"\t"+str(negaposi[pred_classes[i]])+"\t"+str(predictions[i])+"\n")



In [ ]:
### Guided backpropagation ###

imagefolder = "./GBPheatmap_CREs/"

### GBP ###
guidedbackprop_analyzer = GBP(
    partial_model,
    target_id=1,### target "positive"
    relu=True,
)

rate = 0.5 ### threshold value to relative relevance level 0-1
alpha = 1 ### alpha value
transp = "False" ### transparency

for i in range(len(orig_names)):
    example_id = i
    input_img = np.copy(orig_imgs[example_id])
    input_img_1 = np.reshape(input_img,(1,31,4,1))

    # Which class you want to target.
    Xd =array2dna(orig_imgs[example_id])
    target_class = target_classes[example_id]
    pred_class = pred_classes[example_id]
    pred_score = predictions[example_id]
    analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)

    fig = plt.figure(figsize=(20, 8), dpi=300)
    analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)
    heatmap_optional(analysis_guidedbackprop[0].sum(axis=(2)), r=rate, alp=alpha)
    fig.savefig(imagefolder + str(Xd) + ".png", transparent=transp)

